In [5]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [6]:
# Chrome 옵션 설정
options = Options()
options.binary_location = "..\\..\\chrome-win64\\chrome.exe" # chrome 경로 설정
options.add_argument("--no-sandbox") # sandbox 설정
options.add_argument("--headless=new")  # 헤드리스
options.add_argument("--disable-dev-shm-usage") # 공유메모리 설정
options.add_argument("--window-size=1920,1080") # 화면크기설정 , 반응형 웹 대비
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.126 Safari/537.36")
# options.add_argument("--disable-gpu") # GPU 비활성화

# ChromeDriver 설정
service = Service("..\\..\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.opinet.co.kr/")  # https://www.opinet.co.kr/searRgSelect.do
print(driver.title)
time.sleep(4)

# 요소찾기
hover_element = driver.find_element(By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > a")
menu_option = driver.find_element(By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > ul > li:nth-child(1) > a")

# 마우스 호버 엑션
actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()

# # 약간의 시간을 기다린 후 클릭
time.sleep(1)
menu_option.click()
time.sleep(3)
print("주유소검색중")

# 입력창 선택 후 검색
wait = WebDriverWait(driver, 10)
select_element = wait.until(EC.presence_of_element_located((By.ID, "SIGUNGU_NM0")))
select = Select(select_element)
option_count = len(select.options)
time.sleep(3)
# 전체 옵션 반복
try:
    for i in range(1,option_count):
        # 리로딩될 때 셀렉트가 깨지는것 방지
        select_element = wait.until(EC.presence_of_element_located((By.ID, "SIGUNGU_NM0")))
        select = Select(select_element)        
        print("순회시작:",i,'/',option_count - 1)
        time.sleep(2)  # 대기  
        select.select_by_index(i)
        print(f"{i}번 옵션 선택")
        # 옵션 선택 후 상세링크가 나타날 때까지 기다리기
        down = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#templ_list0  div > a")))
        down.click()
        time.sleep(4)  # 대기
except Exception as e:
    print("요소를 못찾았음. 실패...",e)

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [ ]:
service.stop()

In [ ]:
# import os
# os.listdir("C:\\Users\\Admin\\Downloads")

In [ ]:
# 다운받은 파일을 업로드를 위해 옮기기
import os
import shutil
source_folder = "C:\\Users\\Admin\\Downloads\\"
destination_folder = source_folder + "HDFS전송준비"

os.makedirs(destination_folder, exist_ok=True)
for file_name in os.listdir(source_folder):
    if file_name.startswith("지역_위치별(주유소)"):
        src_path = os.path.join(source_folder, file_name)
        dst_path = os.path.join(destination_folder, file_name)
        shutil.move(src_path , dst_path)
        print(f"Moved: {file_name}")

In [ ]:
# 업로드 준비중인 파일을 하둡으로 업로드
from hdfs import InsecureClient
local_folder = "C:\\Users\\Admin\\Downloads\\HDFS전송준비"
hdfs = InsecureClient("http://namenode:9870", user="hadoop")

hdfs_folder = "/user/hadoop/upload_oil"

for file_name in os.listdir(local_folder):
    local_path = os.path.join(local_folder, file_name)
    hdfs_path = hdfs_folder+'/' + file_name
    # print(hdfs_path)
    hdfs.upload(hdfs_path=hdfs_path , local_path=local_path, overwrite= True)
    print(f"업로드 완료: {file_name}")

In [ ]:
hdfs.list('/user/hadoop/')